# Exploratory Data Analysis
https://www.kaggle.com/bkassem/disaster-tweets-eda

# Data Cleaning
https://www.kaggle.com/bkassem/disastor-tweets-cleaning

# Import Libs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMClassifier

import tensorflow as tf
import tensorflow_hub as hub

from transformers import BertTokenizer

print(tf.__version__)

# Read Data

In [ ]:
df = pd.read_csv('../input/disaster-tweets-cleaned/df.csv')
test_df = pd.read_csv('../input/disaster-tweets-cleaned/test_df.csv')
print(df.shape, test_df.shape)

# Plot sentences Lengths

In [ ]:
def plot_sent_len(dataf, col, title):
    dataf['len_' + col] = dataf[col].apply(lambda txt: len(txt.split()))
    plt.hist(dataf['len_' + col], bins = 100)
    plt.title('Train sentences length')
    plt.show()
    return dataf

col = 'ctext'
df = plot_sent_len(df, col, 'sentence lengths')

# Train/Val split

In [ ]:
train_txts, val_txts, y_train, y_val = train_test_split(
    df[col].values, df['target'].values,
    shuffle = True, test_size = 0.15,
    stratify = df['target'].values,
)
test_txts = test_df[col].values
y_test = test_df['target'].values
print('Train size:', train_txts.shape)
print('Validation size:', val_txts.shape)
print('Test size:', test_txts.shape)

# Download Huggingface Tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize Data

In [ ]:
def tokenize_txts(txts, max_len = 40):
    res = tokenizer(
        text = [tokenizer.tokenize(txt) for txt in txts],
        max_length = max_len,
        padding = 'max_length',
        truncation = True,
        is_split_into_words = True,
    )
    return {
        'input_word_ids': res['input_ids'],
        'input_mask': res['attention_mask'],
        'input_type_ids': res['token_type_ids'],
    }

def detokenize_txt(token):
    res = tokenizer.decode(token)
    return res

In [ ]:
MAX_LEN = 35
BATCH_SIZE = 32

In [ ]:
train_tokens = tokenize_txts(train_txts, MAX_LEN)
val_tokens = tokenize_txts(val_txts, MAX_LEN)
test_tokens = tokenize_txts(test_txts, MAX_LEN)


In [ ]:
print('Orginal txt: ', train_txts[0])
print()
sample = train_tokens['input_word_ids'][0]
print('Tokenized txt:', sample)
print()
print('Detokenizd txt:', detokenize_txt(sample))

# Batch the data

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_tokens, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((val_tokens, y_val))
test_ds = tf.data.Dataset.from_tensor_slices((test_tokens, y_test))

train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

# Define Metrics

In [ ]:
class MyF1(tf.keras.metrics.Metric):
    def __init__(self, name = 'mf1_score'):
        super(MyF1, self).__init__(name)
        self.p = tf.metrics.Precision()
        self.r = tf.metrics.Recall()
        self.f1 = self.add_weight(name="f1", initializer="zeros")
    
    def update_state(self, actual, predicted, sample_weight = None):
        self.p.update_state(actual, predicted)
        self.r.update_state(actual, predicted)
        self.f1.assign(2 * self.p.result() * self.r.result() / (self.p.result() + self.r.result()))
        
    def reset_states(self, ):
        self.p.reset_states()
        self.r.reset_states()
        self.f1.assign(0.0)
    def result(self,):
        return self.f1    

# Define Learning rate schedule

In [ ]:
class CSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, lr, freeze_epoch ,batch_size, data_size):
        super(CSchedule, self).__init__()
        self.lr = lr
        self.bs = batch_size
        self.ds = data_size
        self.freeze_epoch = freeze_epoch

    def __call__(self, step):
        epoch = step /(self.ds / self.bs) + 1
        if not self.freeze_epoch or epoch < self.freeze_epoch:
            return self.lr / (tf.cast(epoch, tf.float32)+1)
        else:
            return self.lr * 10 / (tf.cast(epoch, tf.float32)+1)
        
clr = CSchedule(2e-5, 3, BATCH_SIZE, len(train_txts))
plt.plot([clr(x) for x in tf.range(1, 1457, dtype=tf.float32)])
plt.xlabel('steps')
plt.ylabel('learning rate')
plt.title('Learning Rate Schedule')
plt.show()

# Define the Model

In [ ]:
bert_handler = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'

In [ ]:
class MClassifier(tf.keras.Model):
    def __init__(self, dropout_rate):
        super(MClassifier, self).__init__()
        
        self.bert_layer = hub.KerasLayer(
            bert_handler,
            name = 'feature_ext',
            trainable = True,
        )
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.proba = tf.keras.layers.Dense(1, activation = 'sigmoid')

    def call(self, X_in, training):
        x = self.bert_layer(X_in, training = training)
        x = x['pooled_output']
        x = self.dropout(x, training = training)
        output = self.proba(x)
        return output

# Model Trainig

- Fine tune Bert for x epochs
- Freeze Bert after x epochs and train the rest of the network for another y epochs

In [ ]:
LEARNING_RATE = 2e-5
EPOCHS = 5
DP_RATE = 0.3

loss_objective = tf.keras.losses.BinaryCrossentropy()

In [ ]:
def model_evaluation(model, ds, name):
    acc = tf.keras.metrics.BinaryAccuracy()
    f1 = MyF1()
    total_loss = []
    y_hats = []
    for X, y in ds:
        y_hat = model(X, training = False)
        loss = loss_objective(y, y_hat)
        acc.update_state(y, y_hat)
        f1.update_state(y, y_hat)
        total_loss.append(loss.numpy())
        y_hats.append(y_hat)
    y_hats = tf.concat(y_hats, axis = 0).numpy()
    print(f'{name}, loss {np.mean(total_loss):.3f}, acc {acc.result().numpy():.3f}, f1 {f1.result().numpy():.3f}')
    return y_hats, acc.result().numpy(), f1.result().numpy()

In [ ]:
@tf.function
def train_step(model, tr_vars, X, y):
    with tf.GradientTape() as tape:
        y_hat = model(X, training = True)
        loss = loss_objective(y, y_hat)
    grads = tape.gradient(loss, tr_vars)
    return loss, grads, y_hat

def train_model(model, epochs, freeze_bert_on_epoch = None):
    clr = CSchedule(LEARNING_RATE, freeze_bert_on_epoch, BATCH_SIZE, len(train_txts))
    optimizer = tf.keras.optimizers.Adam(clr)
    acc = tf.keras.metrics.BinaryAccuracy()
    f1 = MyF1()
    for epoch in range(1, epochs + 1):
        
        if freeze_bert_on_epoch is not None and epoch == freeze_bert_on_epoch:
            model.get_layer('feature_ext').trainable = False
            
        acc.reset_states()
        f1.reset_states()
        total_loss = []
        for X, y in train_ds:
            tr_vars = model.trainable_variables
            loss, grads, y_hat = train_step(model, tr_vars, X, y)
            optimizer.apply_gradients(zip(grads, tr_vars))
            acc.update_state(y, y_hat)
            f1.update_state(y, y_hat)
            total_loss.append(loss.numpy())
            
        print(f'epoch {epoch}, loss {np.mean(total_loss):.3f}, acc {acc.result().numpy():.3f}, f1 {f1.result().numpy():.3f}')
        model_evaluation(model, val_ds, 'val')
        print()
        
    y_test_hat, _, _ = model_evaluation(model, test_ds, 'test')
    return y_test_hat

In [ ]:
model = MClassifier(DP_RATE,)
y_test_hat = train_model(model, EPOCHS, freeze_bert_on_epoch = 3)

# Model Evaluate on the testset

In [ ]:
y_model_hat = np.array([1 if x[0] >0.5 else 0 for x in y_test_hat])
print(classification_report(y_test, y_model_hat))

# Plot pooled output vs last hidden state

In [ ]:
def plot_embedding(X, y):
    colors = ['green', 'red']
    labels = ['NoDisastor', 'Disastor']
    proj = PCA(2)
    proj_X = proj.fit_transform(X)
    for y_id in np.unique(y):
        plt.scatter(
            x = proj_X[y == y_id, 0], 
            y = proj_X[y == y_id, 1],
            s = 4,
            label = labels[y_id],
            c = colors[y_id], 
            alpha = 0.4
        )
        plt.xlabel('X1')
        plt.ylabel('X2')
        plt.legend()

def plot_embeddings(pooled, seq, mean_max, y, title):
    fig = plt.figure(figsize = (12, 4))
    plt.subplot(1, 3 ,1 )
    plot_embedding(pooled, y)
    plt.title(f'Pooled output in {title} dataset')
    
    plt.subplot(1, 3 ,2)
    plot_embedding(seq, y)
    plt.title(f'Mean hidden state in {title} dataset')
    
    plt.subplot(1, 3 ,3)
    plot_embedding(mean_max, y)
    plt.title(f'MeanMax hidden state in {title} dataset')
    plt.show()
    
def accumulate_embeddings(model, ds):
    embedder = model.get_layer('feature_ext')
    pooled_list = []
    seq_list = []
    for X, y in ds:
        output = embedder(X,)
        pooled_output = output['pooled_output'].numpy()
        seq_output = output['sequence_output'].numpy()
        pooled_list.append(pooled_output)
        seq_list.append(seq_output)
    pooled_list = tf.concat(pooled_list, axis = 0).numpy()
    seq_list = tf.concat(seq_list, axis = 0).numpy()
    return pooled_list, np.mean(seq_list, axis = 1), np.mean(seq_list, axis = 1) + np.max(seq_list, axis = 1)

In [ ]:
tr_pooled, tr_seq, tr_mean_max = accumulate_embeddings(model, train_ds,)
plot_embeddings(tr_pooled, tr_seq,tr_mean_max, y_train, 'Train')

val_pooled, val_seq, val_mean_max = accumulate_embeddings(model, val_ds,)
plot_embeddings(val_pooled, val_seq,val_mean_max, y_val, 'Valid')

test_pooled, test_seq, test_mean_max = accumulate_embeddings(model, test_ds,)
plot_embeddings(test_pooled, test_seq, test_mean_max, y_test, 'Test')

# Train LGBM on the MeanMax hidden state

In [ ]:
def train_gbm_cls(X_tr, y_tr, X_val, y_val, X_test, y_test):
    gbm_cls = LGBMClassifier(
        objective = 'binary',
        class_weight = 'balanced'
    )
    gbm_cls.fit(
        X_tr, y_tr,
        eval_set = (X_val, y_val),
        early_stopping_rounds = 20,
        verbose = 0,
    )
    print('Train')
    print(classification_report(y_train, gbm_cls.predict(X_tr)))
    print('Validation')
    print(classification_report(y_val, gbm_cls.predict(X_val)))
    print('Test')
    gbm_y_hat = gbm_cls.predict(X_test)
    print(classification_report(y_test, gbm_y_hat))
    return gbm_cls, gbm_y_hat

train_feat = tr_mean_max
val_feat = val_mean_max
test_feat = test_mean_max

gbm_cls, gbm_y_hat = train_gbm_cls(train_feat, y_train, val_feat, y_val , test_feat, y_test)

In [ ]:
sub = pd.DataFrame(columns = ['id', 'target'])
sub['id'] = test_df.id
sub['target'] = gbm_y_hat
sub.to_csv('sub.csv', index = False)

In [ ]:
nsub = pd.DataFrame(columns = ['id', 'target'])
nsub['id'] = test_df.id
nsub['target'] = y_model_hat
nsub.to_csv('nsub.csv', index = False)